In [1]:
import pandas as pd
import numpy as np
from scipy.io import loadmat

In [2]:
timesteps = 3511

In [3]:
lfp_buz_features = loadmat('lfp_buz_features_rgs14.mat')

## Bands
- Nx4 data array, where N is the number of epochs, and columns refer to Delta PFC, Theta HPC, Delta/Theta and EMG

In [4]:
bands = np.load('features_matrix_smoothed.npy')

In [5]:
bands.shape

(3511, 6)

## EpochsLinked
- Nx4 data array, where N is the number of epochs, and columns are described as follows:

	- column 1: epoch ID
	- column 2: epoch index (currently not used)
	- column 3: ground truth sleep stage ID, where

				- 1 is associated with wakefulness,
				- 2 is associated with NREM sleep,
				- 3 is associated with REM sleep
	- column 4: the subject ID (used in multi-subject analysis only)

In [6]:
epoch_ids = np.arange(timesteps, dtype=int)
epoch_index = np.arange(timesteps, dtype=int)
states = loadmat('states.mat')
ground_truth_sleep_stage_id = states['downsampledStates'].astype(int)
ground_truth_sleep_stage_id = ground_truth_sleep_stage_id.flatten()[:timesteps]
subject_id = np.ones(timesteps, dtype=int)
epochs_linked = np.column_stack([epoch_ids, 
                                 epoch_index, 
                                 ground_truth_sleep_stage_id,
                                 subject_id
                                ])

## EpochTime
- Nx3 data array, where N is the number of epochs, and columns are described as follows:

	- column 1: epoch ID
	- column 2: recording mode (i.e. baseline or recovery), where
    
			   - 1 is associated with baseline,
			   - 2 is associated with recovery (after sleep deprivation)
	- column 3: the epoch date-time

In [7]:
recording_mode = np.ones(timesteps).astype('<f8')
start_value = 41137.2918055555
step_size = 8.234392028162487e-05
time = np.arange(start_value, start_value + step_size * timesteps, step_size).astype('<f8')
time = time[:timesteps]
epoch_time = np.column_stack([epoch_ids.astype('<f8'), recording_mode, time])

## Save

In [8]:
np.savez("rgs14_smoothed.npz", d=bands, epochsLinked=epochs_linked, epochTime=epoch_time)